
# Time Syncronization of DESI Telemetry Data

As is, the DESI telemetry data is spread out across multiple different tables. As is is shown in the correlations plots in the `correlation` folder, there us very weak correlation across different tables. Before proper analysis can be done, the several datatables of interest must 
be syncronized in time. This involves rebinning and interpolation of the data is currently being hosted by NERSC.

## Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image

## Load Data

In [ ]:
tower_data = pd.read_pickle('../environmentmonitor_tower.pkl')
dome_data = pd.read_pickle('../environmentmonitor_dome.pkl')
telescope_data = pd.read_pickle('../environmentmonitor_telescope.pkl')
etc_seeing_data = pd.read_pickle('../etc_seeing.pkl')
etc_telemetry_data = pd.read_pickle('../etc_telemetry.pkl')
tcs_info_data = pd.read_pickle('../tcs_info.pkl')

## TODO: Sort Data

In [ ]:
# assert(min(tower_data['time_recorded'])==tower_data['time_recorded'][0])
# assert(min(dome_data['time_recorded'])==dome_data['time_recorded'][0])
# assert(min(telescope_data['time_recorded'])==telescope_data['time_recorded'][0])
# assert(min(etc_seeing_data['time_recorded'])==etc_seeing_data['time_recorded'][0])
# assert(min(etc_telemetry_data['time_recorded'])==etc_telemetry_data['time_recorded'][0])
# assert(min(tcs_info_data['time_recorded'])==tcs_info_data['time_recorded'][0])

## Data Visualization Before Syncrization

In [ ]:
display(Image(filename='../correlation/temp_corr_before_sync.png'))

Here, we see strong correlation amongst all the columns related to the dome. However, the mirror-related quantities are not strongly correlated. We would expect the mirror temperature to rise along with the temperature inside the dome. 

In [ ]:
print('initial number of samples per table')
print(f'environmentmonitor_tower: {len(tower_data.index)}')
print(f'environmentmonitor_dome: {len(dome_data.index)}')
print(f'environmentmonitor_telescope: {len(telescope_data.index)}')
print(f'etc_seeing: {len(etc_seeing_data.index)}')
print(f'etc_telemetry: {len(etc_telemetry_data.index)}')
print(f'tcs_info: {len(tcs_info_data.index)}')

In [ ]:
# fig, ax = plt.subplots(ncols=4, figsize=(40,10))

# ax[0].scatter(tower_data['time_recorded'], tower_data['temperature'])
# ax[0].set_xlabel('time_recorded')
# ax[0].set_ylabel('temperature')

# ax[1].scatter(telescope_data['time_recorded'], telescope_data['mirror_avg_temp'])
# ax[1].set_xlabel('time_recorded')
# ax[1].set_ylabel('mirror_avg_temp')

# ax[2].scatter(telescope_data['time_recorded'], telescope_data['mirror_desired_temp'])
# ax[2].set_xlabel('time_recorded')
# ax[2].set_ylabel('mirror_desired_temp')

# ax[3].scatter(etc_telemetry['time_recorded'], etc_telemetry['seeing'])
# ax[3].set_xlabel('time_recorded')
# ax[3].set_ylabel('seeing')

# plt.savefig('seeing_vs_time.png')
# plt.show()